In [1]:
from meta_transformer import torch_utils, module_path, on_cluster
import os
from time import time
from dataclasses import dataclass
import torch

/home/lauro/.virtualenvs/meta-models/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
@dataclass
class Args:
    ndata: int = 23_710
    dataset: str = 'mnist'

args = Args()

In [3]:
print(f'Initial memory allocated: {torch.cuda.memory_allocated() / (1024 ** 2)} MB')
print(f'Initial memory reserved: {torch.cuda.memory_reserved() / (1024 ** 2)} MB')

Initial memory allocated: 0.0 MB
Initial memory reserved: 0.0 MB


In [12]:
if not on_cluster:
    dpath = os.path.join(module_path, "data/david_backdoors")  # local
    # use for testing with small dataset sizes (only works if rds storage is mounted):
    # dpath = os.path.join(module_path, "/home/lauro/rds/model-zoo/")
else:
    dpath = "/rds/user/lsl38/rds-dsk-lab-eWkDxBhxBrQ/model-zoo/"  

model_dataset_paths = {
    "mnist": "mnist-cnns",
    "cifar10": "cifar10",
    "svhn": "svhn",
}

model_dataset_paths = {
    k: os.path.join(dpath, v) for k, v in model_dataset_paths.items()
}

inputs_dirnames = {
    "mnist": "poison_noL1reg",
    #"mnist": "poison",
    #"cifar10": "poison_noL1",
    "cifar10": "poison_easy6_alpha_50",
    "svhn": "poison_noL1",
}


architecture = torch_utils.CNNSmall()  # for CIFAR-10
print("Loading data...")
s = time()
inputs, targets, get_pytorch_model = torch_utils.load_input_and_target_weights(
    model=architecture,
    num_models=args.ndata,
    data_dir=model_dataset_paths[args.dataset],
    inputs_dirname=inputs_dirnames[args.dataset],
    targets_dirname="clean",
)
e = time()

print("Data loading and processing took", round(e - s), "seconds.")

Loading data...
Could not find /home/lauro/projects/meta-models/meta-transformer/data/david_backdoors/mnist-cnns/clean/clean_0000330_0.pt.


In [5]:
print(f'Current memory allocated: {torch.cuda.memory_allocated() / (1024 ** 2)} MB')
print(f'Current memory reserved: {torch.cuda.memory_reserved() / (1024 ** 2)} MB')

Current memory allocated: 0.0 MB
Current memory reserved: 0.0 MB


In [6]:
type(inputs)

numpy.ndarray

In [7]:
type(inputs[0])

dict

In [8]:
inputs[0].keys()

dict_keys(['Conv2d_0', 'Conv2d_1', 'Linear_2', 'Linear_3'])

In [1]:
type(inputs[0]['Conv2d_0']['w'])

NameError: name 'inputs' is not defined